In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
pip install tensorflow-gpu

     |████████████████████████████████| 454.4 MB 10 kB/s 


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [1]:
from google.colab import files
uploaded = files.upload()

Saving eng_-french.csv to eng_-french.csv


In [ ]:
import pandas as pd

In [ ]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['eng_-french.csv']))

In [ ]:
import numpy as np

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pip install keras

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
import codecs
batch_size = 64  
epochs = 2 
latent_dim = 256  
num_samples = 10000

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [ ]:
data=df2

In [ ]:
data.head()
print(data.columns)

Index(['English words/sentences', 'French words/sentences'], dtype='object')


In [ ]:
input_texts=data['English words/sentences']
target_texts=data['French words/sentences']
input_texts=input_texts[:75000]
target_texts=target_texts[:75000]
target_texts=['\t'+text+'\n' for text in target_texts]

In [ ]:
for text in input_texts:
    for char in text:
        input_characters.add(char)

for text in target_texts:
    for char in text:
        target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 75000
Number of unique input tokens: 79
Number of unique output tokens: 105
Max sequence length for inputs: 27
Max sequence length for outputs: 74


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.            
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.    
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(75000, 27, 79)
(75000, 74, 105)
(75000, 74, 105)


In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/2
938/938 [==============================] - 386s 409ms/step - loss: 0.7097 - accuracy: 0.7930 - val_loss: 0.6897 - val_accuracy: 0.7930
Epoch 2/2
938/938 [==============================] - 408s 435ms/step - loss: 0.5019 - accuracy: 0.8495 - val_loss: 0.5805 - val_accuracy: 0.8247


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Elle a mon a cour de conne.

-
Input sentence: Run!
Decoded sentence: La semple est très fait.

-
Input sentence: Run!
Decoded sentence: La semple est très fait.

-
Input sentence: Who?
Decoded sentence: Quel est le mon ?

-
Input sentence: Wow!
Decoded sentence: Nous sommes toutes de toutes.

-
Input sentence: Fire!
Decoded sentence: La semple est très fait.

-
Input sentence: Help!
Decoded sentence: Elle a détent une bien.

-
Input sentence: Jump.
Decoded sentence: La semple est très fait.

-
Input sentence: Stop!
Decoded sentence: La semple est très fait.

-
Input sentence: Stop!
Decoded sentence: La semple est très fait.

-
Input sentence: Stop!
Decoded sentence: La semple est très fait.

-
Input sentence: Wait!
Decoded sentence: Tu es très bien.

-
Input sentence: Wait!
Decoded sentence: Tu es très bien.

-
Input sentence: Go on.
Decoded sentence: Arrête de ne peux !

-
Input sentence: Go on.
Decoded sentence: Arrête de ne peux !

-
Input se

In [ ]:
model.save('model.h5')
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')